In [1]:
import re
from io import StringIO

import open3d as o3d
import numpy as np

class BeaconScanner():
    
    def __init__(self, input_text: str) -> None:
        self.scanners = {}
        
        pattern = r'--- scanner (\d) ---\n((?:-?\d+,-?\d+,-?\d+\n)+)'
        for scanner_s in re.findall(pattern, input_text):
            self.scanners[int(scanner_s[0])] = np.genfromtxt(StringIO(scanner_s[1]), delimiter=',', dtype=int)

    def pcds(self):
        ret = []
        for v in self.scanners.values():
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(scanner.scanners[1])
            ret.append(pcd)
        return ret
        

scanner = BeaconScanner("""--- scanner 0 ---
-618,-824,-621
-537,-823,-458
-447,-329,318
404,-588,-901
544,-627,-890
528,-643,409
-661,-816,-575
390,-675,-793
423,-701,434
-345,-311,381
459,-707,401
-485,-357,347

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
-476,619,847
-460,603,-452
729,430,532
-322,571,750
-355,545,-477
413,935,-424
-391,539,-444
553,889,-390
""")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [28]:
import open3d.pipelines.registration
import open3d.geometry
import copy


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])

    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def preprocess_point_cloud(pcd, voxel_size, downsample=True):
    if downsample:
        print(":: Downsample with a voxel size %.3f." % voxel_size)
        pcd = pcd.voxel_down_sample(voxel_size)
        

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamKNN(10))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamKNN(10))
    return pcd, pcd_fpfh

def prepare_dataset(source, target, voxel_size, downsample):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    # source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
    # target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

    # Why?
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)

    # draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size, downsample)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size, downsample)
    return source_down, target_down, source_fpfh, target_fpfh

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(scanner.scanners[0])

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(scanner.scanners[1])

voxel_size=100
source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pcd, pcd2, voxel_size, downsample=False)

:: Load two point clouds and disturb initial pose.
:: Estimate normal with search radius 200.000.
:: Compute FPFH feature with search radius 500.000.
:: Estimate normal with search radius 200.000.
:: Compute FPFH feature with search radius 500.000.


In [25]:
f1, f2 = [o3d.pipelines.registration.compute_fpfh_feature(
        pcd, o3d.geometry.KDTreeSearchParamKNN(7)).data for p in [pcd, pcd2]]


p1 = f1[:,3].dot(f2)
p2 = np.linalg.norm(f2,axis=0)*np.linalg.norm(f1[:,3])
out1 = p1/p2
# sorted(out1)
sorted([(i, v) for i, v in enumerate(out1)], key=lambda k: k[-1])

for j in range(12):
    print(np.linalg.norm(f1[:,3]-f2[:,j]))

81.33995556555432
94.42818468691861
51.88867898010993
0.0
16.058220143040963
65.12059172369139
83.54326600600882
15.009957994141997
73.34818277359217
75.12764298487234
72.19905785057409
48.32161244947408


In [27]:
out1

array([0.9384296 , 0.91803433, 0.97540242, 1.        , 0.99825748,
       0.96118962, 0.93493029, 0.99798002, 0.95011223, 0.94727804,
       0.95195111, 0.97869705])

In [50]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        # o3d.pipelines.registration.FastGlobalRegistrationOption(
        #     maximum_correspondence_distance=0.025)
    )
    return result

result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
# print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
draw_registration_result(source_down, target_down, result_fast.transformation)

:: Apply fast global registration with distance threshold 50.000


RuntimeError: [1;31m[Open3D Error] (open3d::utility::random::UniformIntGenerator<T>::UniformIntGenerator(T, T) [with T = int]) /root/Open3D/cpp/open3d/utility/Random.h:77: low must be < high, but got low=0 and high=0.
[0;m

In [46]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            # o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
            #     distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 100.000,
   we use a liberal distance threshold 150.000.
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.


In [45]:
result_ransac.transformation

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [44]:
display(np.asarray(source_down.points))
display(np.asarray(source_down.transform(result_ransac.transformation).points))
display(np.asarray(target_down.points))

array([[-621., -618., -824.],
       [-458., -537., -823.],
       [ 318., -447., -329.],
       [-901.,  404., -588.],
       [-890.,  544., -627.],
       [ 409.,  528., -643.],
       [-575., -661., -816.],
       [-793.,  390., -675.],
       [ 434.,  423., -701.],
       [ 381., -345., -311.],
       [ 401.,  459., -707.],
       [ 347., -485., -357.]])

array([[-621., -618., -824.],
       [-458., -537., -823.],
       [ 318., -447., -329.],
       [-901.,  404., -588.],
       [-890.,  544., -627.],
       [ 409.,  528., -643.],
       [-575., -661., -816.],
       [-793.,  390., -675.],
       [ 434.,  423., -701.],
       [ 381., -345., -311.],
       [ 401.,  459., -707.],
       [ 347., -485., -357.]])

array([[ 686.,  422.,  578.],
       [ 605.,  423.,  415.],
       [ 515.,  917., -361.],
       [-336.,  658.,  858.],
       [-476.,  619.,  847.],
       [-460.,  603., -452.],
       [ 729.,  430.,  532.],
       [-322.,  571.,  750.],
       [-355.,  545., -477.],
       [ 413.,  935., -424.],
       [-391.,  539., -444.],
       [ 553.,  889., -390.]])

In [33]:
# from open3d import JVisualizer
from open3d.web_visualizer import draw

colors = [
    (127,201,127),
    (190,174,212),
    (253,192,134),
    (255,255,153),
    (56,108,176),
]
for id_, data in scanner.scanners.items():
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(scanner.scanners[id_])
    pcd.paint_uniform_color([i/256 for i in colors[id_]])
    
    draw(pcd,)
# visualizer = JVisualizer()
# visualizer.add_geometry(pcd)
# visualizer.show()


[Open3D INFO] Window window_29 created.


WebVisualizer(window_uid='window_29')

[Open3D INFO] Window window_30 created.


WebVisualizer(window_uid='window_30')

[Open3D INFO] Window window_31 created.


WebVisualizer(window_uid='window_31')

[Open3D INFO] Window window_32 created.


WebVisualizer(window_uid='window_32')

[Open3D INFO] Window window_33 created.


WebVisualizer(window_uid='window_33')

In [28]:
def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph



In [39]:
print("Full registration ...")
max_correspondence_distance_coarse = 300
max_correspondence_distance_fine = 50
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(scanner.pcds(),
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

Full registration ...
Apply point-to-plane ICP


RuntimeError: [1;31m[Open3D Error] (open3d::pipelines::registration::RegistrationResult open3d::pipelines::registration::RegistrationICP(const open3d::geometry::PointCloud&, const open3d::geometry::PointCloud&, double, const Matrix4d&, const open3d::pipelines::registration::TransformationEstimation&, const open3d::pipelines::registration::ICPConvergenceCriteria&)) /root/Open3D/cpp/open3d/pipelines/registration/Registration.cpp:128: TransformationEstimationPointToPlane and TransformationEstimationColoredICP require pre-computed normal vectors for target PointCloud.
[0;m

[1026:197][83373] (stun_port.cc:308): Port[3856ed00:0:1:0:local:Net[wlp3s0:2001:14ba:6c16:x:x:x:x:x/64:Unknown:id=4]]: UDP send of 70 bytes failed with error 101
[1026:214][83373] (stun_port.cc:308): Port[3824ffc0:0:1:0:local:Net[wlp3s0:2001:14ba:6c16:x:x:x:x:x/64:Unknown:id=4]]: UDP send of 70 bytes failed with error 101
[1026:227][83373] (stun_port.cc:308): Port[38849a70:0:1:0:local:Net[wlp3s0:2001:14ba:6c16:x:x:x:x:x/64:Unknown:id=4]]: UDP send of 22 bytes failed with error 101
[1026:243][83373] (stun_port.cc:308): Port[3811a390:0:1:0:local:Net[wlp3s0:192.168.1.x/24:Unknown:id=3]]: UDP send of 22 bytes failed with error 101
[1026:272][83373] (stun_port.cc:596): sendto : [0x00000065] Network is unreachable
[1026:278][83373] (stun_port.cc:596): sendto : [0x00000065] Network is unreachable
[1026:278][83373] (stun_port.cc:596): sendto : [0x00000065] Network is unreachable
[1026:281][83373] (stun_port.cc:308): Port[3811a390:0:1:0:local:Net[wlp3s0:192.168.1.x/24:Unknown:id=3]]: UDP send o